# Knowledge and Data 2020: Practical Assignment 2
## Manipulate local and external RDF Knowledge Graphs 

YOUR NAME: Vanshita Sharma Kumar

YOUR VUNetID: vkr560

*(If you do not provide your name and VUNetID we will not accept your submission).*

### Learning objectives

At the end of this exercise you should be able to perform some simple manipulations of RDF Data using the rdflib library. You should be able to: 

1. Add and retrieve information from a local RDF database
2. Represent RDF data in other formats, such as the .dot format for graph visualisation
3. Retrieve information from an RDF database created from Web Data
4. Query information from the Web with SPARQL

### Practicalities

Follow this Notebook step-by-step. 

Of course, you can do the exercises in any Programming Editor of your liking. 
But you do not have to. Feel free to simply write code in the Notebook. When 
everything is filled in and works, save the Notebook and submit it 
as a Jupyter Notebook, i.e. with an .ipynb extension. Please use as name of the 
Notebook your studentID+Assignment2.ipynb.  

Other than in courses dedicated to programming we will not evaluate the style
of the programs. But we will test your programs on other data than we provide, 
and your program should give the correct answers to those test-data as well. 

# A. Tasks related to local RDF Knowledge Graphs

This first cell will open a file 'example-from-slide.ttl' using the rdflib library. The first Practical Assignment should have taught you that manipulating symbols as strings is a major pain. 

Programming libraries, such as **rdflib**, help you with this mess once and for all, by parsing the files, creating appropriate datastructures (Graph()) and providing useful functions (such as serialize(), save() and much more). 
Check the website of rdflib http://rdflib.readthedocs.io/: this library does most of the hard work for you.

Before starting with the tasks of this assignment, do not forget to install **rdflib** so we can start using it. Installing libraries in Python is very simple. Just open your terminal and write the following command:

    $ pip install rdflib

For more details on how to install pip and Python libraries, you can check the [preliminaries document](https://github.com/raadjoe/knowledge-data-vu/blob/master/Tutorials/Preliminaries/preliminaries.md).  

In [1]:
from rdflib import Graph, RDF, Namespace, Literal, URIRef

g = Graph()

EX = Namespace('http://example.com/kad2020/')
g.bind('ex',EX)

def serialize():
    #g.serialize() returns a byte string (b'...')
    #.decode() is parsing the byte string into a python3 string
    print(g.serialize(format='turtle'))

def save(filename):
    with open(filename, 'w') as f:
        g.serialize(f, format='nt')
        
def load(filename):
    with open(filename, 'r') as f:
        g.load(f, format='turtle')   

The file 'example-from-slides.ttl' formalises the knowledge base from the slides from Module 1, and a bit more. 

Here is how it looks when you load it into your program and serialise it with rdflib in turtle. 

In [2]:
load('example-from-slides.ttl')
serialize()

@prefix ex: <http://example.com/kad2020/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex:Germany a ex:EuropeanCountry .

ex:Netherlands a ex:Country ;
    ex:has_Capital ex:Amsterdam ;
    ex:has_Name "The Netherlands" ;
    ex:neighbours ex:Belgium .

ex:hasCapital rdfs:range ex:Capital ;
    rdfs:subPropertyOf ex:containsCity .

ex:Amsterdam a ex:Capital .

ex:Belgium a ex:Country .

ex:EuropeanCountry rdfs:subClassOf ex:Country .

ex:containsCity rdfs:domain ex:Country ;
    rdfs:range ex:City .

ex:Capital rdfs:subClassOf ex:City .




Now, we can manipulate the graph very easily, e.g. like in the following very simple function, which returns the predicate(s) that relate a subject to a literal object: 

In [3]:
for s,p,o in g:
    if type(o) == Literal:
        print(p) #p =predicate

http://example.com/kad2020/has_Name


### - Task 1: (1 Point) Add information to an RDF graph

Add 10 triples to the knowledge graph. Make sure that they have the right namespaces. 

Similarily to the triples already present in the file 'example-from-slides.ttl', add at least:
- a new country with its name and capital 
- one triple with a new predicate

Check: http://rdflib.readthedocs.io/en/stable/intro_to_creating_rdf.html

In [4]:
ex = Namespace("http://example.com/kad2020/")
owl = Namespace('http://www.w3.org/2002/07/owl#')

# add 10 triples here to the graph 'g' (do not forget the namespaces).

#g.add((ex.Belgium, RDF.type, ex.country))
g.add((ex.Belgium, ex.has_capital, ex.Brussels))
g.add((ex.Belgium, ex.has_food, ex.chocolate))
g.add((ex.Belgium, ex.has_Name, Literal('Belgium')))


#g.add((ex.EuropeanCountry, RDF.type, ex.country))
g.add((ex.EuropeanCountry, ex.has_country, ex.Netherlands))
g.add((ex.EuropeanCountry, ex.has_country, ex.Germany))
g.add((ex.EuropeanCountry, ex.has_country, ex.Belgium))

#g.add((ex.Germany, RDF.type, ex.country)) # existing relations 
g.add((ex.Germany, ex.has_capital, ex.Berlin))
g.add((ex.Germany, ex.has_Name, Literal('Germany'))) 
g.add((ex.Germany, ex.has_food, ex.Schnitzel))

g.add((ex.Netherlands, ex.has_food, ex.StroopWaffels))

# Literal('Germany')

#g.add((ex.Poland, ex.has_capital, ex.Warsaw))

print(g.serialize(format='turtle')) #.decode("utf-8"))

@prefix ex: <http://example.com/kad2020/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex:hasCapital rdfs:range ex:Capital ;
    rdfs:subPropertyOf ex:containsCity .

ex:Amsterdam a ex:Capital .

ex:EuropeanCountry ex:has_country ex:Belgium,
        ex:Germany,
        ex:Netherlands ;
    rdfs:subClassOf ex:Country .

ex:Germany a ex:EuropeanCountry ;
    ex:has_Name "Germany" ;
    ex:has_capital ex:Berlin ;
    ex:has_food ex:Schnitzel .

ex:Netherlands a ex:Country ;
    ex:has_Capital ex:Amsterdam ;
    ex:has_Name "The Netherlands" ;
    ex:has_food ex:StroopWaffels ;
    ex:neighbours ex:Belgium .

ex:containsCity rdfs:domain ex:Country ;
    rdfs:range ex:City .

ex:Belgium a ex:Country ;
    ex:has_Name "Belgium" ;
    ex:has_capital ex:Brussels ;
    ex:has_food ex:chocolate .

ex:Capital rdfs:subClassOf ex:City .




*After you ran the previous code (adding triples) the next cells will be executed on your extended graph. That is ok.*

### - Task 2a: (1 Point) Get structured information from an RDF graph (all Literals)

Use the functions available in the RDFLib library. Write a small function to print all Literals. 

Hint: there is a function in rdflib to test the type of an object (check previous examples in this notebook)

In [5]:
for s,p,o in g:
    
  #  s = subect 
  #  p = predictae 
  #  o = object 
# g.add(subject, predicate (has_name, has.....), object)
    # Your code here
    if type(o) == Literal:
        print(o) 
    pass

The Netherlands
Germany
Belgium


### - Task 2b: (1 Point) Get structured information from an RDF graph (all unique Predicates)

Please provide another function that gives a **unique** list of the predicates, ordered by occurrence (most occurring first). The answer will look like this: 
<br>http://www.w3.org/2000/01/rdf-schema#label
<br>http://www.w3.org/1999/02/22-rdf-syntax-ns#type
<br>http://example.com/sw2016/locatedIn
<br>http://www.w3.org/2000/01/rdf-schema#range

In [6]:
predicate_dictionary = {}
empty_list = []
for s,p,o in g:
    #print(p)
    if p in predicate_dictionary:
        
        predicate_dictionary[p] += 1
        #print(predicate_dictionary[p])
        
    else:
        predicate_dictionary[p] = 1
        empty_list.append(p)    
    
sorting = sorted(predicate_dictionary.items(), key = lambda x: x[1], reverse=True)
for i in sorting:
    print(i[0])


    # sorted(orders.items(), key=lambda x: x[1], reverse=True)

http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://example.com/kad2020/has_Name
http://example.com/kad2020/has_country
http://example.com/kad2020/has_food
http://example.com/kad2020/has_capital
http://www.w3.org/2000/01/rdf-schema#range
http://www.w3.org/2000/01/rdf-schema#subClassOf
http://example.com/kad2020/has_Capital
http://www.w3.org/2000/01/rdf-schema#subPropertyOf
http://www.w3.org/2000/01/rdf-schema#domain
http://example.com/kad2020/neighbours


# B. Tasks related to Graph visualisations 

### - Task 3a: (1 Point) From RDF to .dot 


In the lecture, we have seen two ways of writing a knowledge graph (simple n-triples, and simple turtle). Let us consider a 3rd syntax, this time a syntax that is useful for visualisation. One standard for visualising graphs is the .dot format.

Print the knowledge graph in .dot file format. Check https://graphviz.gitlab.io/documentation/ for the documentation. You will only need very little of this information, and the most relevant information can be found in the examples that are given. 

<br>Basically, an RDF graph in .dot format starts with 
<br>digraph G { 
    and then a list of links of the following form 
<br>s -> o [label="p"]
    for every (s p o ) in KG (separated by ;
<br>Do not forget to end with a closing bracket. }

An example is 
     
     digraph G { s1 -> o1 [label="p1"] ; s2 -> o2 [label="p2"] } 
     
for an RDF graph {(s1 p1 o1),(s2 p2 o2)}

*You can check how your graph looks like. Just copy & paste your output into: http://www.webgraphviz.com/*

In [7]:
print("digraph G {")
for s,p,o in g:
    # Your code here
    
    s = s.split('/')[-1]
    p= p.split('/')[-1]
    p2 = p.split('#')[-1]
    o = o.split('/')[-1]
        
    # step 1: variable = split it uri 
    # step 2: index 
    
    if p2:
        print(f'{s} -> {o} [label="{p2}"] ')
    else:
        print(f'{s} -> {o} [label="{p}"] ') 
    #print(s,p,o,p2)
    pass

print("}")

digraph G {
Germany -> Berlin [label="has_capital"] 
Netherlands -> The Netherlands [label="has_Name"] 
Germany -> EuropeanCountry [label="type"] 
EuropeanCountry -> Netherlands [label="has_country"] 
containsCity -> City [label="range"] 
Belgium -> chocolate [label="has_food"] 
Netherlands -> StroopWaffels [label="has_food"] 
Germany -> Germany [label="has_Name"] 
Netherlands -> Country [label="type"] 
Netherlands -> Amsterdam [label="has_Capital"] 
hasCapital -> containsCity [label="subPropertyOf"] 
Belgium -> Belgium [label="has_Name"] 
Belgium -> Country [label="type"] 
containsCity -> Country [label="domain"] 
EuropeanCountry -> Germany [label="has_country"] 
Netherlands -> Belgium [label="neighbours"] 
Belgium -> Brussels [label="has_capital"] 
Capital -> City [label="subClassOf"] 
EuropeanCountry -> Country [label="subClassOf"] 
Amsterdam -> Capital [label="type"] 
hasCapital -> Capital [label="range"] 
Germany -> Schnitzel [label="has_food"] 
EuropeanCountry -> Belgium [label="

### - Task 3b: (1 Point) From RDF to .dot with "semantic information"

There is a conceptual distinction between properties, instances and classes (sets of instances). A simple way of checking is the following

1. in a triple (s a o), with predicate a (which is a special abbreviation for the predicate rdf:type), the s is an Instance, and o is a Class. 
2. in a triple (s rdfs:subClassOf o) both s and o are Classes. 
3. in a triple (p rdfs:domain o) p is a Property and o is a Class. 
4. in a triple (p rdfs:range o)  p is a Property and o is a Class. 

Make a .dot representation for an RDF graph that distinguishes between types of links (RDF vocabulary vs others) and types of nodes (Classes versus Instances) via different colors. 

*You can check how your graph looks like. Just copy & paste your output into: http://www.webgraphviz.com/*

In [8]:
print("digraph G {")

instances = "[color: Green]"
classes = "[color: Blue]"
properties = "[color: Orange]"

for s,p,o in g:
    # Your code here
    
    s = s.split('/')[-1]
    p= p.split('/')[-1].split('#')[-1]
    #p2 = p.split('#')[-1]
    o = o.split('/')[-1]
        
    if p == 'a': 
        print(s +" "+instances)
        print(o +" "+ classes)
        # [label="has_capital", color = [red]] 
    elif p == 'subClassOf':
        print(s +" "+ instances)
        print(o +" "+ classes)
    elif p == 'domain':
        print(s +" "+ properties)
        print(o +" "+ classes)
    elif p== 'range':
        print(s +" "+ properties) # predicate range is a property
        print(o +" "+ classes)
    #print(s,p,o,p2)
    #if p2:
    #    print(f'{s} -> {o} [label="{p2}"] ')
    #else:
    print(f'{s} -> {o} [label="{p}"]')
    
print("}")

digraph G {
Germany -> Berlin [label="has_capital"]
Netherlands -> The Netherlands [label="has_Name"]
Germany -> EuropeanCountry [label="type"]
EuropeanCountry -> Netherlands [label="has_country"]
containsCity [color: Orange]
City [color: Blue]
containsCity -> City [label="range"]
Belgium -> chocolate [label="has_food"]
Netherlands -> StroopWaffels [label="has_food"]
Germany -> Germany [label="has_Name"]
Netherlands -> Country [label="type"]
Netherlands -> Amsterdam [label="has_Capital"]
hasCapital -> containsCity [label="subPropertyOf"]
Belgium -> Belgium [label="has_Name"]
Belgium -> Country [label="type"]
containsCity [color: Orange]
Country [color: Blue]
containsCity -> Country [label="domain"]
EuropeanCountry -> Germany [label="has_country"]
Netherlands -> Belgium [label="neighbours"]
Belgium -> Brussels [label="has_capital"]
Capital [color: Green]
City [color: Blue]
Capital -> City [label="subClassOf"]
EuropeanCountry [color: Green]
Country [color: Blue]
EuropeanCountry -> Countr

### - Task 4a: (1 Point) Deriving implicit knowledge (a bit of schema)

We will look into Schema information in the latter modules, but let us try already to find some implicit information in a first bit of inferencing: whenever there are two statements (s a o) and (o rdfs:subClassOf o2) we can derive (and later prove) that (s a o2). 

Write a procedure that adds all implied triples to our knowledge graph. 

In [9]:
subclass = {}
for s,p,o in g:
    
    if p in RDF.type:
        subclass[o] = s
        
print(subclass)

for s,p,o in g:
    
    if p in 'subClassOf':
        if s in subclass.keys():
            advanced_s = subclass.get(s)
            advanced_o = o
            
            g.add(advanced_s, RDF.type, advanced_o)
            print(advanced_s, advanced_o + "new")
            

print(g.serialize(format='turtle'))
    

{rdflib.term.URIRef('http://example.com/kad2020/EuropeanCountry'): rdflib.term.URIRef('http://example.com/kad2020/Germany'), rdflib.term.URIRef('http://example.com/kad2020/Country'): rdflib.term.URIRef('http://example.com/kad2020/Belgium'), rdflib.term.URIRef('http://example.com/kad2020/Capital'): rdflib.term.URIRef('http://example.com/kad2020/Amsterdam')}
@prefix ex: <http://example.com/kad2020/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex:hasCapital rdfs:range ex:Capital ;
    rdfs:subPropertyOf ex:containsCity .

ex:Amsterdam a ex:Capital .

ex:EuropeanCountry ex:has_country ex:Belgium,
        ex:Germany,
        ex:Netherlands ;
    rdfs:subClassOf ex:Country .

ex:Germany a ex:EuropeanCountry ;
    ex:has_Name "Germany" ;
    ex:has_capital ex:Berlin ;
    ex:has_food ex:Schnitzel .

ex:Netherlands a ex:Country ;
    ex:has_Capital ex:Amsterdam ;
    ex:has_Name "The Netherlands" ;
    ex:has_food ex:StroopWaffels ;
    ex:neighbours ex:Belgium .

ex:containsCit

### - Task 4b: (Optional - 1 Extra Point) Visualising implicit knowledge

Produce a .dot version of the graph with those implies implicit triples, and mark the edges of those triples with a different color or arrow style. 

In [10]:
# Your code here

# C. Tasks related to local copies of external RDF Datasets using SPARQL

Until now, we have manipulated local knowledge graphs, but as we claimed in the lectures, the advantage of knowledge graphs is that they can easily be linked with other datasets on the Web. 

In the remaining 3 tasks, we will manipulate data from the Web, and ask complex queries over this web data. 

In the first task, we will access web data, make a local copy of it, and then query it. In the other two tasks, we will query live data directly from web Knowledge Graphs (in this case, the SPARQL endpoint of DBPedia). 

### - Task 5: (1 Point) Show and manipulate data about RDF resources on the Web 

With rdflib we can easily load a graph, even if it comes from a source on the Web. The following snupped loads as graph the information about the resource Amsterdam from Dbpedia.

In [15]:
import requests

endpoint = "https://dbpedia.org/sparql"
query = 'DESCRIBE <http://dbpedia.org/resource/Amsterdam>'

payload = {'query':query, 'format':'text/turtle'}
response = requests.post(endpoint, data = payload)

g = Graph()
g.parse(data=response.text, format='ttl')

<Graph identifier=Nbb06c73183634ca8be58036e84dd3cc1 (<class 'rdflib.graph.Graph'>)>

In [16]:
query = 'DESCRIBE <http://dbpedia.org/resource/Rotterdam>'

payload = {'query':query, 'format':'text/turtle'}
response = requests.post(endpoint, data = payload)

g.parse(data=response.text, format='ttl')  # calling parse again merges the graphs

<Graph identifier=Nbb06c73183634ca8be58036e84dd3cc1 (<class 'rdflib.graph.Graph'>)>

Let us start by showing diverse bits of information w.r.t  Amsterdam and Rotterdam in DBPedia. It is very similar to task 1, but now with Web Data. 

First, query the graph g (now containing the DBPedia information about Amsterdam and Rotterdam) and check whether you can find someone who was born in Amsterdam (is dbo:birthPlace of) and died in Rotterdam (is dbo:deathPlace of)?

In [17]:
qres = g.query(
   """
    PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT ?s
        WHERE {
            ?s dbo:birthPlace dbr:Amsterdam .
            ?s dbo:deathPlace dbr:Rotterdam .
        }
        LIMIT 10
       """)
for row in qres:
    print("%s" % row)

http://dbpedia.org/resource/Haya_van_Someren
http://dbpedia.org/resource/Gabriël_Vigeveno
http://dbpedia.org/resource/Jan_Stolker
http://dbpedia.org/resource/Nans_van_Leeuwen


Write a query to check whether there is an album that was recorded both in Rotterdam and Amsterdam? You need to look at the data to know which property you should check for. 

To get an intuition of what is in the knowledge graph you might want to look at the human readable rendering on : http://dbpedia.org/resource/Amsterdam

In [14]:
qres = g.query(
   """
    PREFIX dbr: <http://dbpedia.org/resource/>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    SELECT ?s  
        WHERE {
            ?s dbo:recordedIn dbr:Amsterdam .
            ?s dbo:recordedIn dbr:Rotterdam .
        }
        LIMIT 10
       """)
for row in qres:
    print("%s" % row)

### - Task 6: (2 Points) Ask SPARQL against live data using Yasgui

Yasgui (http://yasgui.org/) is a nice graphical interface for asking queries.

Run a new query against http://dbpedia.org/sparql that does the following:

- Find all languages spoken in countries that are not official languages of that country
- The query should return two colums: the country, and the number of languages.
- Order the countries by the number of unofficial languages, from high to low.

### - Task 7: (1 Point) SPARQL 

Write a SPARQL query which returns all relationship(s) between the series "The Office (UK)" and the actor "Ricky Gervais" (literally).

Be careful to check for relations in both directions (but not necessarily the same relation in both directions).  

Use Yasgui to design the correct SPARQL query, and copy paste it in the cell below. 

PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX dbo: <http://dbpedia.org/ontology/>

SELECT DISTINCT ?show_information
WHERE {
  { <http://dbpedia.org/resource/The_Office_(British_TV_series)> ?show_information dbr:Ricky_Gervais . }
  UNION
  { dbr:Ricky_Gervais ?show_information <http://dbpedia.org/resource/The_Office_(British_TV_series)> . }
}